In [2]:
import numpy as np
import pandas as pd
import calendar
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import classification_report,confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,LSTM,Embedding
from keras import optimizers
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
# preprocess=pd.read_csv('preprocess.csv', engine='python',index_col="arrival_date")
train=pd.read_csv('preprocess_all/preprocess_train_includecancel_per_all.csv', engine='python',index_col="arrival_date")
# train=train.drop(["label"],axis=1)
train

,hotel_Resort,hotel_City,lead_time,arrival_date_week_number,stays_nights,adults,children,babies,meal_BB,meal_FB,...,wait_less_than_3,wait_between_3_30,wait_greater_than_30,contract,group,transient,transient_party,mean_required_car_parking_space,special_requests_ratio,quatity
arrival_date,,,,,,,,,,,,,,,,,,,,,
2015/7/1,0.352459,0.647541,186.745902,27,3.114754,1.827869,0.016393,0.000000,0.368852,0.008197,...,1.000000,0.000000,0.0,0.049180,0.000000,0.934426,0.016393,0.032787,0.221311,122
2015/7/2,0.473118,0.526882,153.129032,27,3.935484,2.000000,0.043011,0.000000,0.892473,0.010753,...,1.000000,0.000000,0.0,0.365591,0.000000,0.387097,0.247312,0.053763,0.301075,93
2015/7/3,0.714286,0.285714,72.857143,27,4.285714,2.000000,0.071429,0.000000,0.642857,0.000000,...,1.000000,0.000000,0.0,0.125000,0.000000,0.642857,0.232143,0.125000,0.375000,56
2015/7/4,0.568182,0.431818,154.965909,27,4.988636,1.943182,0.090909,0.022727,0.863636,0.011364,...,1.000000,0.000000,0.0,0.079545,0.000000,0.500000,0.420455,0.079545,0.386364,88
2015/7/5,0.849057,0.150943,83.641509,28,5.962264,1.943396,0.188679,0.000000,0.716981,0.000000,...,1.000000,0.000000,0.0,0.188679,0.018868,0.754717,0.037736,0.245283,0.339623,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017/3/27,0.409836,0.590164,92.825137,13,2.830601,1.846995,0.010929,0.000000,0.885246,0.000000,...,1.000000,0.000000,0.0,0.000000,0.010929,0.737705,0.251366,0.043716,0.469945,183
2017/3/28,0.488636,0.511364,61.102273,13,4.022727,1.761364,0.034091,0.011364,0.795455,0.000000,...,1.000000,0.000000,0.0,0.000000,0.011364,0.965909,0.022727,0.045455,0.431818,88
2017/3/29,0.383117,0.616883,44.642857,13,2.837662,1.675325,0.051948,0.012987,0.798701,0.000000,...,1.000000,0.000000,0.0,0.000000,0.019481,0.967532,0.012987,0.084416,0.461039,154


In [4]:
y_label=pd.read_csv('train_label.csv', engine='python',index_col="arrival_date")
# test_nolabel = pd.read_csv('test_nolabel.csv', engine='python')
y_label_tr=pd.get_dummies(y_label, columns=["label"])
y_label_tr

,label_0.0,label_1.0,label_2.0,label_3.0,label_4.0,label_5.0,label_6.0,label_7.0,label_8.0,label_9.0
arrival_date,,,,,,,,,,
2015-07-01,0,0,1,0,0,0,0,0,0,0
2015-07-02,0,1,0,0,0,0,0,0,0,0
2015-07-03,0,1,0,0,0,0,0,0,0,0
2015-07-04,0,1,0,0,0,0,0,0,0,0
2015-07-05,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2017-03-27,0,0,1,0,0,0,0,0,0,0
2017-03-28,0,1,0,0,0,0,0,0,0,0
2017-03-29,0,0,1,0,0,0,0,0,0,0


In [5]:
# preprocess=pd.read_csv('preprocess.csv', engine='python',index_col="arrival_date")
test=pd.read_csv('preprocess_all/preprocess_test_per_all.csv', engine='python',index_col="arrival_date")
# train=train.drop(["label"],axis=1)
test

,hotel_Resort,hotel_City,lead_time,arrival_date_week_number,stays_nights,adults,children,babies,meal_BB,meal_FB,...,wait_less_than_3,wait_between_3_30,wait_greater_than_30,contract,group,transient,transient_party,mean_required_car_parking_space,special_requests_ratio,quatity
arrival_date,,,,,,,,,,,,,,,,,,,,,
2017/4/1,0.265152,0.734848,95.265152,13,3.969697,2.068182,0.151515,0.007576,0.803030,0.000000,...,0.977273,0.000000,0.022727,0.007576,0.000000,0.803030,0.189394,0.053030,0.409091,132
2017/4/2,0.302083,0.697917,73.364583,14,3.822917,1.947917,0.083333,0.010417,0.760417,0.000000,...,0.989583,0.000000,0.010417,0.041667,0.000000,0.916667,0.041667,0.052083,0.520833,96
2017/4/3,0.277056,0.722944,76.995671,14,3.307359,1.761905,0.082251,0.004329,0.878788,0.000000,...,0.991342,0.000000,0.008658,0.017316,0.012987,0.757576,0.212121,0.038961,0.415584,231
2017/4/4,0.224000,0.776000,104.544000,14,3.048000,1.912000,0.152000,0.000000,0.888000,0.000000,...,0.984000,0.016000,0.000000,0.000000,0.000000,0.888000,0.112000,0.056000,0.360000,125
2017/4/5,0.361111,0.638889,119.769841,14,3.484127,1.702381,0.051587,0.000000,0.920635,0.000000,...,0.980159,0.003968,0.015873,0.003968,0.000000,0.527778,0.468254,0.067460,0.190476,252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017/8/27,0.385057,0.614943,93.954023,35,3.178161,1.948276,0.258621,0.000000,0.660920,0.011494,...,1.000000,0.000000,0.000000,0.017241,0.011494,0.867816,0.103448,0.068966,0.643678,174
2017/8/28,0.369668,0.630332,139.222749,35,4.374408,1.696682,0.151659,0.004739,0.478673,0.014218,...,1.000000,0.000000,0.000000,0.018957,0.009479,0.630332,0.341232,0.056872,0.450237,211
2017/8/29,0.280000,0.720000,141.672000,35,3.888000,1.816000,0.168000,0.000000,0.744000,0.000000,...,1.000000,0.000000,0.000000,0.016000,0.016000,0.952000,0.016000,0.048000,0.600000,125


In [6]:
y_test_label=pd.read_csv('test_nolabel.csv', engine='python')
# test_nolabel = pd.read_csv('test_nolabel.csv', engine='python')
# y_label_tr=pd.get_dummies(y_label, columns=["label"])
y_test_label

,arrival_date
0,2017-04-01
1,2017-04-02
2,2017-04-03
3,2017-04-04
4,2017-04-05
...,...
148,2017-08-27
149,2017-08-28
150,2017-08-29
151,2017-08-30


In [10]:
train_lstm=np.array(train)
# train_lstm=train.drop(["arrival_date"],axis=1)
train_lstm=np.array(train_lstm)
X_train = np.reshape(train_lstm, (train_lstm.shape[0], 1, train_lstm.shape[1]))
X_train, X_test, y_train, y_test = train_test_split(X_train, y_label_tr, test_size=0.2, random_state=42)
model = Sequential()
#     model.add(LSTM(units = 256, input_dim=42))
model.add(LSTM(units = 256, input_dim=42))
# model.add(LSTM(128))#new
# model.add(LSTM(64))#new
# model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
#     model.add(Dropout(0.25))
#     model.add(Dense(16))
model.add(Dense(10, activation='sigmoid'))
callback = EarlyStopping(monitor="val_loss", patience=20, verbose=1, mode="auto")
model.compile(loss='binary_crossentropy',optimizer='adam')
model.fit(X_train, y_train, batch_size=200, epochs=1000, verbose=0,callbacks=[callback])
#     scores = model.evaluate(X_test, y_test, verbose=0)
# #     print("%s: %.2f%%" % (model.metrics_names[1], scores[1]))
#     cvscores.append(scores[1] * 100)

y_pred_train = model.predict(X_test, batch_size=64, verbose=0)
y_pred_bool_test = np.argmax(y_pred_train, axis=1)
y_test_bool = np.argmax(np.array(y_test), axis=1)
print(classification_report(y_test_bool, y_pred_bool_test))
print(confusion_matrix(y_test_bool, y_pred_bool_test))

              precision    recall  f1-score   support

           0       0.16      1.00      0.27        20
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        34
           3       0.00      0.00      0.00        27
           4       0.00      0.00      0.00        10
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         1

    accuracy                           0.16       128
   macro avg       0.02      0.12      0.03       128
weighted avg       0.02      0.16      0.04       128

[[20  0  0  0  0  0  0  0]
 [31  0  0  0  0  0  0  0]
 [34  0  0  0  0  0  0  0]
 [27  0  0  0  0  0  0  0]
 [10  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0]]


In [32]:
error = []
for i in range(len(y_test_bool)):
    error.append(y_test_bool[i] - y_pred_bool_test[i])

# print("Errors: ", error)
# print(error)

absError = []
for val in error:
    absError.append(abs(val))#誤差絕對值

# print("Absolute Value of Error: ", absError)
from math import sqrt
print("MAE = ", sum(absError) / len(absError))#平均絕對誤差MAE
 

MAE =  2.0


In [33]:
train_lstm=np.array(train)
# train_lstm=train.drop(["arrival_date"],axis=1)
# train_lstm=np.array(train_lstm)
X_train = np.reshape(train_lstm, (train_lstm.shape[0], 1, train_lstm.shape[1]))
# X_train, X_test, y_train, y_test = train_test_split(X_train, y_label_tr, test_size=0.2, random_state=42)
model = Sequential()
#     model.add(LSTM(units = 256, input_dim=42))
model.add(LSTM(units = 256, input_dim=42))
# model.add(LSTM(128))#new
# model.add(LSTM(64))#new
# model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
#     model.add(Dropout(0.25))
#     model.add(Dense(16))
model.add(Dense(10, activation='sigmoid'))
callback = EarlyStopping(monitor="val_loss", patience=20, verbose=1, mode="auto")
model.compile(loss='binary_crossentropy',optimizer='adam')
model.fit(X_train, y_label_tr, batch_size=200, epochs=1000, verbose=0,callbacks=[callback])
#     scores = model.evaluate(X_test, y_test, verbose=0)
# #     print("%s: %.2f%%" % (model.metrics_names[1], scores[1]))
#     cvscores.append(scores[1] * 100)

y_pred_train = model.predict(X_train, batch_size=64, verbose=0)
y_pred_bool_train = np.argmax(y_pred_train, axis=1)
y_train_bool = np.argmax(np.array(y_label_tr), axis=1)
print(classification_report(y_train_bool, y_pred_bool_train))
print(confusion_matrix(y_train_bool, y_pred_bool_train))

              precision    recall  f1-score   support

           0       0.15      1.00      0.26        94
           1       0.00      0.00      0.00       152
           2       0.00      0.00      0.00       186
           3       0.00      0.00      0.00       124
           4       0.00      0.00      0.00        46
           5       0.00      0.00      0.00        21
           6       0.00      0.00      0.00        13
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         1

    accuracy                           0.15       640
   macro avg       0.01      0.10      0.03       640
weighted avg       0.02      0.15      0.04       640

[[ 94   0   0   0   0   0   0   0   0   0]
 [152   0   0   0   0   0   0   0   0   0]
 [186   0   0   0   0   0   0   0   0   0]
 [124   0   0   0   0   0   0   0   0   0]
 [ 46   0   0   0   0   0   0   0   0   0]
 [ 21   0   0   0   0   0 

In [34]:
test_lstm=np.array(test)
X_test = np.reshape(test_lstm, (test_lstm.shape[0], 1, test_lstm.shape[1]))
y_pred_test = model.predict(X_test, batch_size=64, verbose=0)
y_pred_bool_test = np.argmax(y_pred_test, axis=1)
y_pred_bool_test=pd.DataFrame(y_pred_bool_test,columns=["label"])
y_test_num=pd.concat([y_test_label,y_pred_bool_test],axis=1)

In [8]:
y_test_num

,arrival_date,label
0,2017-04-01,3
1,2017-04-02,3
2,2017-04-03,9
3,2017-04-04,3
4,2017-04-05,4
...,...,...
148,2017-08-27,6
149,2017-08-28,6
150,2017-08-29,4
151,2017-08-30,3


In [9]:
import csv

# 開啟輸出的 CSV 檔案
with open('model_predict/lstm_per.csv', 'w', newline='') as csvfile:
    # 建立 CSV 檔寫入器
    writer = csv.writer(csvfile)

    # 寫入標題資料
    writer.writerow(["arrival_date","label"])
    # 寫入距離資料
    for i in range(len(y_test_num)):
        writer.writerow([y_test_num["arrival_date"].iloc[i],y_test_num["label"].iloc[i]])
        

In [10]:
y_test_num.groupby("label").count()

,arrival_date
label,
2,1
3,24
4,40
5,5
6,52
8,30
9,1


In [11]:
y_label_full=pd.read_csv('train_label.csv', engine='python')

In [12]:
y_label_full.groupby("label").count()

,arrival_date
label,
0.0,94
1.0,152
2.0,186
3.0,124
4.0,46
5.0,21
6.0,13
7.0,1
8.0,2
